In [1]:
import sys
sys.path.insert(0, '..')
from eda_import import *

In [2]:
%time adf = pd.read_csv('../__results/aca_pval_with_sc_info.csv')
bdf = adf.query('has_diff_aca == True').copy()

CPU times: user 59.4 ms, sys: 15.9 ms, total: 75.3 ms
Wall time: 73.3 ms


In [3]:
bdf.head(2)

,disease,gene_name,mkid,mclv,strand,aclv,N_on,T_on,N_off,T_off,fisher_exact_p,N_base,T_base,N_on_ratio,T_on_ratio,N2T_ratio_diff,diff_is_significant,N2T_ratio_change,N2T_ratio_change_sig,has_diff_aca,sc_list,src_list,num_sc
74,BLCA,BRCA1,chr17|BRCA1|-|41196318,41196318,-,41196313,13,401,6,13,0.000059,19,414,0.684211,0.968599,0.284389,True,up,up,True,41197695,protein_coding,1
75,BLCA,BRCA1,chr17|BRCA1|-|41196822,41196822,-,41196822,15,323,4,91,1.000000,19,414,0.789474,0.780193,-0.009280,False,down,NaN,True,41197801,protein_coding,1


In [4]:
bdf['chr'] = bdf.mkid.apply(lambda x: x.split('|')[0])

In [5]:
bdf.head(2)

,disease,gene_name,mkid,mclv,strand,aclv,N_on,T_on,N_off,T_off,fisher_exact_p,N_base,T_base,N_on_ratio,T_on_ratio,N2T_ratio_diff,diff_is_significant,N2T_ratio_change,N2T_ratio_change_sig,has_diff_aca,sc_list,src_list,num_sc,chr
74,BLCA,BRCA1,chr17|BRCA1|-|41196318,41196318,-,41196313,13,401,6,13,0.000059,19,414,0.684211,0.968599,0.284389,True,up,up,True,41197695,protein_coding,1,chr17
75,BLCA,BRCA1,chr17|BRCA1|-|41196822,41196822,-,41196822,15,323,4,91,1.000000,19,414,0.789474,0.780193,-0.009280,False,down,NaN,True,41197801,protein_coding,1,chr17


In [8]:
clv_sc_df = pd.read_csv(
    '../__results/annotated-clv-sc-mapping.csv',
    usecols=['seqname', 'gene_name', 'strand', 'source', 'aclv', 'sc']
)
clv_sc_df.drop_duplicates(inplace=True)
clv_sc_df.rename(columns={'aclv': 'clv'}, inplace=True)
clv_sc_df.reset_index(drop=True, inplace=True)

## Print summary and detail tables

In [9]:
def summarize_annot_clv_sc(grp):
#     assert grp.offset.unique().shape[0] == 1
#     offset = grp.offset.unique()[0]
    
    scs  = list(sorted(grp.sc.unique()))
    clvs = list(sorted(grp.clv.unique()))
    mapped_pairs = [tuple(_) for _ in grp[['clv', 'sc']].sort_values(['clv', 'sc']).values]
    res = pd.Series([clvs, np.diff(clvs), len(clvs), scs, np.diff(scs), len(scs), 
                     mapped_pairs, len(mapped_pairs)],
                    index=['gtf_clv', 'neigh_dist_gtf_clv', 'num_gtf_clv',
                           'gtf_sc', 'neigh_dist_gtf_sc', 'num_gtf_sc',
                           'gtf_mapped_pairs', 'num_gtf_mapped_pairs'])
    return res

In [10]:
annot_sum = clv_sc_df.groupby('gene_name').apply(summarize_annot_clv_sc).reset_index()

In [11]:
bdf.head(1)

,disease,gene_name,mkid,mclv,strand,aclv,N_on,T_on,N_off,T_off,fisher_exact_p,N_base,T_base,N_on_ratio,T_on_ratio,N2T_ratio_diff,diff_is_significant,N2T_ratio_change,N2T_ratio_change_sig,has_diff_aca,sc_list,src_list,num_sc,chr
74,BLCA,BRCA1,chr17|BRCA1|-|41196318,41196318,-,41196313,13,401,6,13,0.000059,19,414,0.684211,0.968599,0.284389,True,up,up,True,41197695,protein_coding,1,chr17


In [12]:
def sum_apa_pval(grp):
    dises = list(sorted(grp.disease.unique()))
    clvs =  list(sorted(grp.mclv.unique()))
    res = pd.Series([dises, len(dises), clvs, np.diff(clvs), len(clvs)], 
                    index=['dise', 'num_dise', 'exp_clv', 'neigh_dist_exp_clv', 'num_exp_clv'])
    return res

In [13]:
sum_df = bdf.groupby(['gene_name', 'chr', 'strand']).apply(sum_apa_pval).reset_index()

In [14]:
sum_df = sum_df.merge(annot_sum, on='gene_name')

In [15]:
# reorder columns if necessary
sum_df.columns.values
# sum_df = sum_df[[]]

array(['gene_name', 'chr', 'strand', 'dise', 'num_dise', 'exp_clv',
       'neigh_dist_exp_clv', 'num_exp_clv', 'gtf_clv',
       'neigh_dist_gtf_clv', 'num_gtf_clv', 'gtf_sc', 'neigh_dist_gtf_sc',
       'num_gtf_sc', 'gtf_mapped_pairs', 'num_gtf_mapped_pairs'], dtype=object)

Check a few cases

In [16]:
sum_df.query('gene_name == "CDKN2A"')

,gene_name,chr,strand,dise,num_dise,exp_clv,neigh_dist_exp_clv,num_exp_clv,gtf_clv,neigh_dist_gtf_clv,num_gtf_clv,gtf_sc,neigh_dist_gtf_sc,num_gtf_sc,gtf_mapped_pairs,num_gtf_mapped_pairs
7,CDKN2A,chr9,-,"[COAD, HNSC, KICH, KIRC, KIRP, LIHC, PRAD, THCA]",8,"[21967755, 21967992, 21968200, 21968733, 21969564, 21970697, 21994884]","[237, 208, 533, 831, 1133, 24187]",7,"[21967752, 21967995, 21968055, 21968104, 21968180, 21968723, 21969568, 21970715]","[243, 60, 49, 76, 543, 845, 1147]",8,"[21968208, 21968228, 21968724, 21969731, 21970890, 21971002, 21974476]","[20, 496, 1007, 1159, 112, 3474]",7,"[(21967752, 21968228), (21967752, 21971002), (21967995, 21968208), (21967995, 21968228), (21967995, 21971002), (21968055, 21968724), (21968104, 21968228), (21968180, 21968228), (21968180, 21974476), (21968723, 21968724), (21969568, 21969731), (21970715, 21970890)]",12


In [17]:
len(sum_df.query('gene_name == "CDKN2A"').gtf_clv.values[0])

8

CDKN2A has 7 clv sites (after clustering, 12 before), but there are (clv, sc) pairs, so at at least one clv is used twice

In [18]:
sum_df.query('gene_name == "EZH2"')

,gene_name,chr,strand,dise,num_dise,exp_clv,neigh_dist_exp_clv,num_exp_clv,gtf_clv,neigh_dist_gtf_clv,num_gtf_clv,gtf_sc,neigh_dist_gtf_sc,num_gtf_sc,gtf_mapped_pairs,num_gtf_mapped_pairs
11,EZH2,chr7,-,"[BRCA, KIRC, KIRP, LIHC, LUAD, PRAD, THCA]",7,"[148504477, 148504561, 148514434, 148515148, 148526831]","[84, 9873, 714, 11683]",5,"[148504477, 148504574, 148504617, 148515129]","[97, 43, 10512]",4,"[148504738, 148516131, 148529833]","[11393, 13702]",3,"[(148504477, 148504738), (148504477, 148516131), (148504574, 148504738), (148504617, 148504738), (148515129, 148516131), (148515129, 148529833)]",6


In [19]:
sum_df.query('num_gtf_sc == 1').shape

(5, 16)

So 5 genes with significant shift are one-stop-codon genes

format some columns for good print

In [20]:
sum_df.columns.values

array(['gene_name', 'chr', 'strand', 'dise', 'num_dise', 'exp_clv',
       'neigh_dist_exp_clv', 'num_exp_clv', 'gtf_clv',
       'neigh_dist_gtf_clv', 'num_gtf_clv', 'gtf_sc', 'neigh_dist_gtf_sc',
       'num_gtf_sc', 'gtf_mapped_pairs', 'num_gtf_mapped_pairs'], dtype=object)

In [21]:
sum_df_pretty = sum_df.copy()
for col in [
    'dise',
    'exp_clv', 'neigh_dist_exp_clv',
    'gtf_clv', 'neigh_dist_gtf_clv',
    'gtf_sc', 'neigh_dist_gtf_sc',
    'gtf_mapped_pairs',
]:
    print(col)
    sum_df_pretty[col] = sum_df_pretty[col].apply(lambda l: ', '.join(map(str, l)))

dise
exp_clv
neigh_dist_exp_clv
gtf_clv
neigh_dist_gtf_clv
gtf_sc
neigh_dist_gtf_sc
gtf_mapped_pairs


Mannually adding trends

In [22]:
# trends = [
#     ['CCNE1', 'lengthening'],
#     ['CDKN2C', 'lengthening'],
#     ['FGF2', 'shortening'],
#     ['KIT', 'shortening'],
#     ['MAP2K1', 'lengthening'],
#     ['REL', 'lengthening'],
# ]
# trends_df = pd.DataFrame(trends, columns=['gene_name', 'trend'])

# sum_df_pretty.shape

# sum_df_pretty = sum_df_pretty.merge(trends_df, on='gene_name', how='outer').fillna('undefined')

In [23]:
sum_df_pretty.shape

(33, 16)

In [24]:
renaming_dd = {
    'gene_name': 'Gene',
    'strand': 'Strand',
    'chr': 'Chromosome',
#     'trend': 'Trend',
    
    'dise': 'Diseases with differential CS usage patterns',
    'num_dise': '# diseases',
    
    'exp_clv': 'Predicted CSs',
    'neigh_dist_exp_clv': 'Distance between neighboring predicted CSs',
    'num_exp_clv': '# predicted CSs',

    'gtf_clv': 'annotated CSs (after clustering)',
    'neigh_dist_gtf_clv': 'Distance between neighboring annotated CSs',
    'num_gtf_clv': '# annotated CSs',

    'gtf_sc': 'annotated stop codons',
    'neigh_dist_gtf_sc': 'Distance between neighboring annotated stop codons',
    'num_gtf_sc': '# annotated stop codons',

    'gtf_mapped_pairs': 'annotated CS-stop codon pairs',
    'num_gtf_mapped_pairs': '# annotated CS-stop codon pairs',
}

In [25]:
sum_df_pretty.rename(columns=renaming_dd, inplace=True)

In [26]:
sum_df_pretty.to_csv('../__results/clv_sites_freq_shift_summary.tsv', sep='\t', index=False)

# Also format the details table and save to disk

### mapp predicted CSs to annotated CSs

In [27]:
# first create a dict of gene-to-clvs map
ANNOT_GENE2CLV_DD = clv_sc_df.groupby(['gene_name']).apply(lambda g: g.clv.unique())

In [28]:
ANNOT_GENE2CLV_DD['BRCA1']

array([41196313, 41197694, 41251848, 41196822, 41197646, 41246129, 41197580])

In [29]:
def map_kclv2aclv(row):
    """map KLEAT predicted clv to a closest annotated clv"""
    gene = row.gene_name
    kclv = row['mclv']
    poss_aclvs = ANNOT_GENE2CLV_DD[gene]
    return poss_aclvs[np.argmin(np.abs(poss_aclvs - kclv))]

In [30]:
dad_df = bdf.copy()

In [31]:
dad_df['aclv'] = dad_df.apply(map_kclv2aclv, axis=1)

In [32]:
dad_df['aclv'].values[:10]

array([41196313, 41196822, 41197694, 41197694, 41246129, 41246129,
       41246129, 41246129, 41251848, 41251848])

In [33]:
dad_df['dist_kclv2aclv'] = np.abs(dad_df['mclv'].values - dad_df['aclv'].values)

In [34]:
KCLV2ACLV_CUTOFF = 25

In [35]:
dad_df['dist_kclv2acl_lt_25bp'] = dad_df['dist_kclv2aclv'] < KCLV2ACLV_CUTOFF

In [36]:
ANNOT_CLV2SC_DD = clv_sc_df.groupby(['gene_name', 'clv']).apply(lambda g: g.sc.unique()).to_dict()

In [37]:
# mapped_adf[['gene_name', 'aclv']].apply(lambda row: (row.gene_name, row.aclv), axis=1)

def map_aclv2sc(row):
    """with aclv assigned, now add the corresponding annotated stop codon information"""
    # ('gene_name', 'aclv') acts like a shorter kid (skid)
    skid = (row.gene_name, row.aclv)
    scs = ANNOT_CLV2SC_DD[skid]
    return scs.tolist()

In [38]:
# asc: annotated stop codons
dad_df['asc_list'] = dad_df[['gene_name', 'aclv']].apply(map_aclv2sc, axis=1)

In [39]:
dad_df['num_asc'] = dad_df['asc_list'].apply(len)

In [40]:
dad_df.shape

(576, 28)

In [41]:
dad_df.num_asc.value_counts()

1    476
2    88 
3    10 
4    2  
Name: num_asc, dtype: int64

In [42]:
dad_df.query('num_asc > 1').shape[0] / dad_df.shape[0]

0.1736111111111111

so about 17.4% of cleavage sites are mapped to multiple stop codons

In [43]:
def calc_utr_len(row):
    if row.dist_kclv2acl_lt_25bp and row.num_asc == 1:
        return abs(row.mclv - row.asc_list[0])
    else:
        return 'NA'

In [44]:
dad_df['utr_len'] = dad_df.apply(calc_utr_len, axis=1)

In [45]:
dad_df.has_diff_aca.value_counts()

True    576
Name: has_diff_aca, dtype: int64

In [46]:
dad_df.sample(2)

,disease,gene_name,mkid,mclv,strand,aclv,N_on,T_on,N_off,T_off,fisher_exact_p,N_base,T_base,N_on_ratio,T_on_ratio,N2T_ratio_diff,diff_is_significant,N2T_ratio_change,N2T_ratio_change_sig,has_diff_aca,sc_list,src_list,num_sc,chr,dist_kclv2aclv,dist_kclv2acl_lt_25bp,asc_list,num_asc,utr_len
1347,BRCA,NF1,chr17|NF1|+|29687999,29687999,+,29687975,60,420,54,719,0.001186,114,1139,0.526316,0.368745,-0.157571,True,down,down,True,29687729,protein_coding,1,chr17,24,True,[29687729],1,270
7037,LUSC,CHURC1,chr14|CHURC1|+|65411291,65411291,+,65411310,19,163,32,341,0.531610,51,504,0.372549,0.323413,-0.049136,False,down,NaN,True,65410039,protein_coding,1,chr14,19,True,[65410039],1,1252


## Now write out the detailed table for each predicted CS

In [47]:
# also used for sorting
sort_cols = ['gene_name', 'chr', 'disease', 'mclv', 'strand']

In [48]:
diff_ppd_pretty = dad_df.copy()\
    .drop(['N2T_ratio_change_sig', 'has_diff_aca', 'mkid'], axis=1)\
    .rename(columns={'N_base': 'N_total', 'T_base': 'T_total'})
diff_ppd_pretty = diff_ppd_pretty[
        sort_cols + 
    ['N_on', 'T_on', 'N_off', 'T_off', 'N_total', 'T_total', 
     'N_on_ratio', 'T_on_ratio', 'N2T_ratio_change', 'fisher_exact_p'] +
    # mapped annotated CS and SC
    ['aclv', 'dist_kclv2aclv', 'dist_kclv2acl_lt_25bp', 'asc_list', 'num_asc', 'utr_len']
]


diff_ppd_pretty['N_on_ratio'] = diff_ppd_pretty['N_on_ratio'].apply(lambda x: '{0:.2f}'.format(x))
diff_ppd_pretty['T_on_ratio'] = diff_ppd_pretty['T_on_ratio'].apply(lambda x: '{0:.2f}'.format(x))
diff_ppd_pretty['p-value < 0.01'] = diff_ppd_pretty.fisher_exact_p < 0.01

diff_ppd_pretty['asc_list'] = diff_ppd_pretty['asc_list'].apply(lambda l: ', '.join(map(str, sorted(l))))
diff_ppd_pretty['utr_len'] = diff_ppd_pretty['utr_len'].apply(lambda x: '{0:.0f}'.format(x) if x != 'NA' else x)
diff_ppd_pretty.sort_values(sort_cols, inplace=True)

In [49]:
diff_ppd_pretty.head(2)

,gene_name,chr,disease,mclv,strand,N_on,T_on,N_off,T_off,N_total,T_total,N_on_ratio,T_on_ratio,N2T_ratio_change,fisher_exact_p,aclv,dist_kclv2aclv,dist_kclv2acl_lt_25bp,asc_list,num_asc,utr_len,p-value < 0.01
810,AKT2,chr19,BRCA,40736224,-,105,878,9,261,114,1139,0.92,0.77,down,0.000068,40736224,0,True,40739779,1,3555,True
811,AKT2,chr19,BRCA,40736624,-,13,40,101,1099,114,1139,0.11,0.04,down,0.000545,40736224,400,False,40739779,1,NA,True


In [50]:
raw = ['N_on', 'T_on', 'N_off', 'T_off', 'N_total', 'T_total', 'N_on_ratio', 'T_on_ratio']

In [51]:
# remove underscore
renaming_dd = dict(zip(raw, [_.replace('_', ' ') for _ in raw]))

In [52]:
renaming_dd.update({
        'gene_name': 'Gene',
        'chr': 'Chromosome',
        'strand': 'Strand',
        'disease': 'Disease',
        'mclv': 'CS',
        'N2T_ratio_change': 'ratio change from N to T',
        'fisher_exact_p': 'p-value',
        
        'aclv': 'mapped annotated CS candidate',
        'dist_kclv2aclv': 'distance between predicted and annotated CS',
        'dist_kclv2acl_lt_25bp': 'distance < 25 bp',
        'asc_list': 'mapped annotated stop codon(s)',
        'num_asc': '# mapped stop codons',
        'utr_len': "3'UTR length"
})

In [53]:
# convert to human readable column names
diff_ppd_pretty.rename(columns=renaming_dd, inplace=True)

In [54]:
diff_ppd_pretty.head(2)

,Gene,Chromosome,Disease,CS,Strand,N on,T on,N off,T off,N total,T total,N on ratio,T on ratio,ratio change from N to T,p-value,mapped annotated CS candidate,distance between predicted and annotated CS,distance < 25 bp,mapped annotated stop codon(s),# mapped stop codons,3'UTR length,p-value < 0.01
810,AKT2,chr19,BRCA,40736224,-,105,878,9,261,114,1139,0.92,0.77,down,0.000068,40736224,0,True,40739779,1,3555,True
811,AKT2,chr19,BRCA,40736624,-,13,40,101,1099,114,1139,0.11,0.04,down,0.000545,40736224,400,False,40739779,1,NA,True


In [55]:
# put two p-value related columns together
cols = diff_ppd_pretty.columns.values.tolist()
_c = 'p-value < 0.01'
cols.remove(_c)
cols.insert(cols.index('p-value') + 1, _c)

In [57]:
diff_ppd_pretty[cols].to_csv('../__results/clv_sites_freq_shift_details.tsv', sep='\t', index=False)